In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series

In [2]:
Call = pd.read_csv('./data/_dcebfb2135a2bf5a6392493bd61aba22_detroit-311.csv', low_memory=False)
Call



,ticket_id,city,issue_type,ticket_status,issue_description,rating,ticket_closed_date_time,acknowledged_at,ticket_created_date_time,ticket_last_updated_date_time,address,lat,lng,location,image
0,1516722,City of Detroit,Clogged Drain,Acknowledged,"Two drains one on each side of street, street ...",3,NaN,03/06/2015 10:03:38 PM,03/06/2015 09:57:52 PM,04/12/2015 01:01:10 AM,"13120-13130 Ilene Street Detroit, MI 48238, USA",42.383998,-83.161039,"(42.3839977668, -83.1610385642)",NaN
1,1525361,City of Detroit,Clogged Drain,Acknowledged,standing water on lumplin,2,NaN,03/11/2015 04:23:11 PM,03/11/2015 04:14:29 PM,04/07/2015 02:04:44 PM,"1485 E. Outer Drive Detroit, Michigan",42.440471,-83.080919,"(42.4404708, -83.080919)",NaN
2,1525218,City of Detroit,Clogged Drain,Closed,CITZEN CALLED TO REPORT CLOGGED DRAINS,2,08/15/2015 12:03:43 AM,03/11/2015 03:39:05 PM,03/11/2015 03:26:20 PM,08/15/2015 12:03:44 AM,"15460 Eastburn Detroit, Michigan",42.445244,-82.962038,"(42.445244, -82.962038)",NaN
3,1525214,City of Detroit,Clogged Drain,Acknowledged,Citizen called DWSD to report clogged drain,3,NaN,03/11/2015 03:35:02 PM,03/11/2015 03:22:42 PM,06/07/2015 10:07:48 PM,"17541 Mendota St Detroit, Michigan",42.421043,-83.166194,"(42.421043, -83.166194)",NaN
4,1525142,City of Detroit,Clogged Drain,Acknowledged,@ THE CORNER OF GRIGGS & MARGARETA,2,NaN,03/11/2015 03:04:59 PM,03/11/2015 02:53:23 PM,03/11/2015 03:04:59 PM,"Griggs Detroit, Michigan",42.402033,-83.162874,"(42.4020334, -83.1628741)",NaN
5,1525087,City of Detroit,Clogged Drain,Closed,NaN,2,08/24/2015 01:51:47 PM,03/11/2015 02:54:57 PM,03/11/2015 02:41:16 PM,08/24/2015 01:51:47 PM,"14902 Kentucky Detroit, Michigan",42.399431,-83.158100,"(42.399431, -83.1581)",NaN
6,1524994,City of Detroit,Clogged Drain,Acknowledged,Citizen called to report a blocked basin locat...,2,NaN,03/11/2015 02:28:52 PM,03/11/2015 02:16:24 PM,04/27/2015 01:17:08 PM,"20089 Vaughan Detroit, Michigan",42.439669,-83.240740,"(42.439669, -83.24074)",NaN
7,1524775,City of Detroit,Clogged Drain,Acknowledged,Citizen called DWSD to report clogged drain,2,NaN,03/11/2015 01:14:42 PM,03/11/2015 01:09:28 PM,03/11/2015 01:14:42 PM,"18663 Fenelon St Detroit, Michigan",42.430693,-83.053367,"(42.4306931, -83.0533671)",NaN
8,1524652,City of Detroit,Clogged Drain,Acknowledged,CCitizen called DWSD to report clogged drain -...,2,NaN,03/11/2015 12:54:39 PM,03/11/2015 12:42:18 PM,03/11/2015 12:54:39 PM,"16170 Ilene St Detroit, Michigan",42.410764,-83.161803,"(42.410764, -83.161803)",NaN
9,1524630,City of Detroit,Clogged Drain,Closed,Citzen called DWSD to report Clogged Drain,2,08/22/2015 12:08:19 AM,03/11/2015 12:44:38 PM,03/11/2015 12:35:22 PM,08/22/2015 12:08:19 AM,"19530 Roselawn St Detroit, Michigan",42.437114,-83.152779,"(42.437114, -83.152779)",NaN


In [3]:
calls = sorted({cr for cr in Call['issue_type']})
orederedcalls = { val : idx for idx, val in enumerate(sorted(calls))} 
calls

['Abandoned Vehicle',
 'Clogged Drain',
 'Curbside Solid Waste Issue',
 'Customer Service (internal use only, private issue)',
 'DPW - Debris Removal',
 'DPW - Other environmental',
 'Detroit Land Bank Referral',
 'Fire Hydrant Issue',
 'Graffiti',
 'Graffiti Abatement (internal use only, public issue)',
 'Illegal Dumping / Illegal Dump Sites',
 'Manhole Cover Issue',
 'Potholes',
 'Residential Snow Removal Issue',
 'Running Water in a Home or Building',
 'Street Light Pole Down',
 'Test (internal use only, public issue)',
 'Traffic Sign Issue',
 'Traffic Signal Issue',
 'Trash Issue - Bulk waste deposited more than 24 hours before designated time',
 'Trash Issue - Improper placement of refuse container between collections/left at curbside',
 'Tree Issue',
 'Water Main Break']

In [7]:
import sqlite3

query = "insert into CallType (Name) values ('{}')"
    
con = sqlite3.connect("./data/location.s3db")
cur = con.cursor()
    
for call in calls:
    cur.execute(query.format(call))
con.commit()
con.close()

In [6]:
import sqlite3
import dateutil.parser

def Insert(row, idx, cur):  
    name = 'Call311{}'.format(idx+1)
    cur.execute("select Id from LocationReferenced where OriginId='{}'".format(name))
    idx = cur.fetchone()
    if idx is None:
        return
    index = idx[0]
    date = None
    rawdate = row['ticket_created_date_time']
    try:
        date = dateutil.parser.parse(rawdate)
    except:
        print(rawdate)
        pass
    indextype = orederedcalls[row['issue_type']]+1
    rating = row['rating']
    info ={'index':index, 'date':date, 'indextype': indextype, 'rating' : rating}
    cur.execute('insert into CallData (LocationReferencedId, CallTypeId, Date, Rating) values (:index,:indextype,:date,:rating)', info)

con = sqlite3.connect("./data/location.s3db")
cur = con.cursor()

for index, row in Call.iterrows():
    Insert(row, index, cur)
con.commit()
con.close()